In [2]:
# imports
import pandas as pd
import requests
#import time

# For optional visualization later
#import matplotlib.pyplot as plt
#import seaborn as sns

# Display options for DataFrames
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 10) 

# Foursquare

In [3]:
import os

import requests

# Get the API key from the environment

api_key = os.environ['FOURSQUARE_API_KEY']


# use the API key to make the request
url = "https://api.foursquare.com/v3/places/search"

# Example parameters (you can adjust based on your use case)
params = {
    'll': '48.888559,2.264200',  # Example: coordinates for a specific location
    'radius': 1000,  # Search radius in meters
    'limit': 10,  # Limit to 5 results
}

# Setup headers including the API key
headers = {
    "accept": "application/json",
    "Authorization": api_key  # Include the API key in the request header
}

response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Results:")
    # Loop through and print each result
    for place in data['results']:
        print(f"Name: {place['name']}")
        print(f"Address: {place.get('location', {}).get('address', 'N/A')}")
        print(f"Distance: {place['distance']} meters")
        print("-" * 40)
else:
    print(f"Error fetching data from Foursquare: {response.status_code}")
    


Results:
Name: Les Pieds dans l'Eau
Address: 39 boulevard du Parc
Distance: 448 meters
----------------------------------------
Name: Maison Wok
Address: 4 rue des Poissonniers
Distance: 494 meters
----------------------------------------
Name: Livio
Address: 6 rue de Longchamp
Distance: 626 meters
----------------------------------------
Name: Le Nouveau Paris
Address: 204 boulevard Bineau
Distance: 700 meters
----------------------------------------
Name: Cercle Nautique de France
Address: Ile du Pont de Neuilly
Distance: 397 meters
----------------------------------------
Name: Bristrot Quai
Address: 6 boulevard du Général Leclerc
Distance: 552 meters
----------------------------------------
Name: Vietnam Kitchen
Address: 10 rue de Bezons
Distance: 946 meters
----------------------------------------
Name: Monoprix
Address: 72-80 avenue du Général de Gaulle
Distance: 927 meters
----------------------------------------
Name: Katsura
Address: 17 rue Sainte-Marie
Distance: 773 meters
--

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [1]:
import os
import requests
import pandas as pd

# Load the DataFrame from the CSV file (ensure it's the correct path)
df_bike_stations = pd.read_csv('paris_bike_stations.csv')

# Check the first few rows of the DataFrame to confirm it's loaded correctly
print(df_bike_stations.head())

# Get the API key from the environment
api_key = os.environ['FOURSQUARE_API_KEY']

# Set up the base URL for the Foursquare API
url = "https://api.foursquare.com/v3/places/search"

# Function to get nearby places for a given bike station (latitude, longitude)
def get_nearby_places(lat, lon):
    params = {
        'll': f'{lat},{lon}',  # Latitude and Longitude as a string in the format "lat,lon"
        'radius': 1000,        # Search radius in meters
        'limit': 5,            # Limit to 5 results
    }

    # Setup headers including the API key
    headers = {
        "accept": "application/json",
        "Authorization": api_key  # Include the API key in the request header
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        return response.json()  # Return the JSON response
    else:
        print(f"Error fetching data from Foursquare for {lat}, {lon}: {response.status_code}")
        return None

# Create a list to store the results from Foursquare
foursquare_results = []

# Iterate over each bike station in your dataframe (assuming it's Paris Velib)
for index, row in df_bike_stations.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    station_name = row['name']
    
    # Get nearby places for the bike station
    nearby_places = get_nearby_places(lat, lon)
    
    if nearby_places:
        for place in nearby_places.get('results', []):
            foursquare_results.append({
                'station_name': station_name,
                'venue_name': place['name'],
                'venue_address': place.get('location', {}).get('address', 'N/A'),
                'venue_distance': place.get('distance', 'N/A'),
                'venue_latitude': place.get('location', {}).get('lat', 'N/A'),
                'venue_longitude': place.get('location', {}).get('lng', 'N/A'),
            })

# Create a DataFrame for the Foursquare results
df_foursquare = pd.DataFrame(foursquare_results)

# Display the first few rows of the Foursquare results
print(df_foursquare.head())

# Optionally save the results as a CSV
df_foursquare.to_csv('foursquare_nearby_places.csv', index=False)


                                name   latitude  longitude  free_bikes
0                 Argenson - Château  48.888559   2.264200           4
1             Saint Lambert - Blomet  48.836591   2.293056          25
2            Saint-Maur - République  48.864186   2.378317          16
3          Mairie de Rosny-sous-Bois  48.871257   2.486581          27
4  Mouton Duvernet - Général Leclerc  48.831633   2.329312           5
         station_name                 venue_name           venue_address  \
0  Argenson - Château       Les Pieds dans l'Eau    39 boulevard du Parc   
1  Argenson - Château                 Maison Wok  4 rue des Poissonniers   
2  Argenson - Château                      Livio      6 rue de Longchamp   
3  Argenson - Château           Le Nouveau Paris    204 boulevard Bineau   
4  Argenson - Château  Cercle Nautique de France  Ile du Pont de Neuilly   

   venue_distance venue_latitude venue_longitude  
0             448            N/A             N/A  
1             4

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [2]:
import os
import requests
import pandas as pd

# Load the DataFrame from the CSV file (ensure it's the correct path)
df_bike_stations = pd.read_csv('paris_bike_stations.csv')

# Check the first few rows of the DataFrame to confirm it's loaded correctly
print(df_bike_stations.head())

# Get the API key from the environment
api_key = os.environ['FOURSQUARE_API_KEY']

# Set up the base URL for the Foursquare API
url = "https://api.foursquare.com/v3/places/search"

# Function to get nearby places for a given bike station (latitude, longitude)
def get_nearby_places(lat, lon):
    params = {
        'll': f'{lat},{lon}',  # Latitude and Longitude as a string in the format "lat,lon"
        'radius': 1000,        # Search radius in meters
        'limit': 5,            # Limit to 5 results
    }

    # Setup headers including the API key
    headers = {
        "accept": "application/json",
        "Authorization": api_key  # Include the API key in the request header
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        return response.json()  # Return the JSON response
    else:
        print(f"Error fetching data from Foursquare for {lat}, {lon}: {response.status_code}")
        return None

# Create a list to store the results from Foursquare
foursquare_results = []

# Iterate over each bike station in your dataframe (assuming it's Paris Velib)
for index, row in df_bike_stations.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    station_name = row['name']
    
    # Get nearby places for the bike station
    nearby_places = get_nearby_places(lat, lon)
    
    if nearby_places:
        for place in nearby_places.get('results', []):
            # Extract the venue details
            venue_name = place.get('name', 'N/A')
            venue_address = place.get('location', {}).get('address', 'N/A')
            venue_lat = place.get('location', {}).get('lat', 'N/A')
            venue_lon = place.get('location', {}).get('lng', 'N/A')
            venue_rating = place.get('rating', 'N/A')  # Rating of the place
            venue_categories = [category['name'] for category in place.get('categories', [])]  # List of categories
            
            # Add the venue information to the results
            foursquare_results.append({
                'station_name': station_name,
                'venue_name': venue_name,
                'venue_address': venue_address,
                'venue_rating': venue_rating,
                'venue_categories': ', '.join(venue_categories),  # Join categories in a string
                'venue_distance': place.get('distance', 'N/A'),
                'venue_latitude': venue_lat,
                'venue_longitude': venue_lon,
            })



#save the results as a CSV
df_foursquare.to_csv('foursquare_nearby_places.csv', index=False)


                                name   latitude  longitude  free_bikes
0                 Argenson - Château  48.888559   2.264200           4
1             Saint Lambert - Blomet  48.836591   2.293056          25
2            Saint-Maur - République  48.864186   2.378317          16
3          Mairie de Rosny-sous-Bois  48.871257   2.486581          27
4  Mouton Duvernet - Général Leclerc  48.831633   2.329312           5


Put your parsed results into a DataFrame

In [13]:
# Create a DataFrame for the Foursquare results
df_foursquare = pd.DataFrame(foursquare_results)

# Display the first few rows of the Foursquare results
print(df_foursquare.head())

         station_name                 venue_name           venue_address  \
0  Argenson - Château       Les Pieds dans l'Eau    39 boulevard du Parc   
1  Argenson - Château                 Maison Wok  4 rue des Poissonniers   
2  Argenson - Château                      Livio      6 rue de Longchamp   
3  Argenson - Château           Le Nouveau Paris    204 boulevard Bineau   
4  Argenson - Château  Cercle Nautique de France  Ile du Pont de Neuilly   

  venue_rating                                   venue_categories  \
0          N/A                                  French Restaurant   
1          N/A                                    Thai Restaurant   
2          N/A  Pizzeria, Fast Food Restaurant, Italian Restau...   
3          N/A                            Café, French Restaurant   
4          N/A                              Sports and Recreation   

   venue_distance venue_latitude venue_longitude  
0             448            N/A             N/A  
1             494         

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [19]:
import os
import requests
import time
import csv
import pandas as pd

# Load bike stations data from CSV
df_bike_stations = pd.read_csv('paris_bike_stations.csv')

# Yelp API key from environment
yeld_api_key = os.environ['YELD_API_KEY']

# Yelp API endpoint
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Set headers with API key
headers = {
    "Authorization": f"Bearer {yeld_api_key}"
}

# Function to get Yelp results near a coordinate
def get_yelp_places(lat, lon):
    params = {
        "latitude": lat,
        "longitude": lon,
        "radius": 1000,  # 1000 meters
        "limit": 5       # Limit to 5 results
    }

    response = requests.get(yelp_url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Yelp API error for {lat},{lon}: {response.status_code}")
        return None

# Open a CSV file to write results
with open('yelp_nearby_places.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['station_name', 'business_name', 'rating', 'address', 'distance'])

    # Iterate over a limited number of bike stations for faster execution
    for index, row in df_bike_stations.head(30).iterrows():
        lat = row['latitude']
        lon = row['longitude']
        station_name = row['name']

        print(f"\nNearby places for: {station_name}")
        results = get_yelp_places(lat, lon)

        if results:
            for business in results.get("businesses", []):
                writer.writerow([
                    station_name,
                    business.get('name'),
                    business.get('rating', 'N/A'),
                    ', '.join(business['location'].get('display_address', [])),
                    business.get('distance', 'N/A')
                ])

        # Commented out to speed up execution (use only if within rate limits)
        # time.sleep(0.5)

print("Yelp results saved to 'yelp_nearby_places.csv'")



Nearby places for: Argenson - Château

Nearby places for: Saint Lambert - Blomet

Nearby places for: Saint-Maur - République

Nearby places for: Mairie de Rosny-sous-Bois

Nearby places for: Mouton Duvernet - Général Leclerc

Nearby places for: Pernety - Raymond Losserand

Nearby places for: Champs de Manoeuvre - Cartoucherie

Nearby places for: Belleville - Télégraphe

Nearby places for: Abbeville - Faubourg Poissonnière

Nearby places for: Porte de Pantin.

Nearby places for: Passy - Place du Costa Rica

Nearby places for: Miromesnil - La Boétie

Nearby places for: Place Etienne Pernet

Nearby places for: Bons Raisins - Voltaire

Nearby places for: Saint-Augustin

Nearby places for: Lecourbe - Jean Maridor

Nearby places for: Colonel Monteil - Brune

Nearby places for: République - Auguste Perret

Nearby places for: Marcel Pourtout - Édouard Manet 

Nearby places for: Port Royal - Hôpital Cochin

Nearby places for: Adolphe Pinard - Victor Hugo

Nearby places for: Claude Decaen - Can

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [21]:
import os
import requests
import time
import csv
import pandas as pd

# Load bike stations data from CSV
df_bike_stations = pd.read_csv('paris_bike_stations.csv')

# Yelp API key from environment
yeld_api_key = os.environ['YELD_API_KEY']

# YelD API endpoint
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Set headers with API key
headers = {
    "Authorization": f"Bearer {yeld_api_key}"
}

# Function to get Yelp results near a coordinate
def get_yelp_places(lat, lon):
    params = {
        "latitude": lat,
        "longitude": lon,
        "radius": 1000,  # 1000 meters
        "limit": 5       # Limit to 5 results
    }

    response = requests.get(yelp_url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Yelp API error for {lat},{lon}: {response.status_code}")
        return None

# Open a CSV file to write results
with open('yelp_nearby_places.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write header row, including category
    writer.writerow(['station_name', 'business_name', 'rating', 'address', 'distance', 'category'])

    # Limit to 30 stations to reduce execution time
    for index, row in df_bike_stations.head(30).iterrows():
        lat = row['latitude']
        lon = row['longitude']
        station_name = row['name']

        print(f"\nNearby places for: {station_name}")
        results = get_yelp_places(lat, lon)

        if results:
            for business in results.get("businesses", []):
                # ----- PARSING START -----
                name = business.get('name', 'N/A')
                rating = business.get('rating', 'N/A')
                address = ', '.join(business['location'].get('display_address', []))
                distance = business.get('distance', 'N/A')
                categories = business.get('categories', [])
                category_names = [cat.get('title', '') for cat in categories]
                primary_category = category_names[0] if category_names else 'N/A'
                # ----- PARSING END -----

                writer.writerow([
                    station_name,
                    name,
                    rating,
                    address,
                    distance,
                    primary_category
                ])

        time.sleep(0.5)  # Uncomment to respect rate limits

print("Yelp POI results saved to 'yelp_nearby_places.csv'")



Nearby places for: Argenson - Château

Nearby places for: Saint Lambert - Blomet

Nearby places for: Saint-Maur - République

Nearby places for: Mairie de Rosny-sous-Bois

Nearby places for: Mouton Duvernet - Général Leclerc

Nearby places for: Pernety - Raymond Losserand

Nearby places for: Champs de Manoeuvre - Cartoucherie

Nearby places for: Belleville - Télégraphe

Nearby places for: Abbeville - Faubourg Poissonnière

Nearby places for: Porte de Pantin.

Nearby places for: Passy - Place du Costa Rica

Nearby places for: Miromesnil - La Boétie

Nearby places for: Place Etienne Pernet

Nearby places for: Bons Raisins - Voltaire

Nearby places for: Saint-Augustin

Nearby places for: Lecourbe - Jean Maridor

Nearby places for: Colonel Monteil - Brune

Nearby places for: République - Auguste Perret

Nearby places for: Marcel Pourtout - Édouard Manet 

Nearby places for: Port Royal - Hôpital Cochin

Nearby places for: Adolphe Pinard - Victor Hugo

Nearby places for: Claude Decaen - Can

Put your parsed results into a DataFrame

In [22]:
import os
import requests
import pandas as pd

# Load bike stations
df_bike_stations = pd.read_csv('paris_bike_stations.csv')

# Yelp API key from environment
yeld_api_key = os.environ['YELD_API_KEY']

# Yelp API endpoint
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Set headers with API key
headers = {
    "Authorization": f"Bearer {yeld_api_key}"
}

# Function to get Yelp results near a coordinate
def get_yelp_places(lat, lon):
    params = {
        "latitude": lat,
        "longitude": lon,
        "radius": 1000,  # 1000 meters
        "limit": 5       # Limit to 5 results
    }

    response = requests.get(yelp_url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Yelp API error for {lat},{lon}: {response.status_code}, {response.text}")
        return None

parsed_results = []

# Iterate over bike stations and retrieve data
for index, row in df_bike_stations.head(30).iterrows():
    lat = row['latitude']
    lon = row['longitude']
    station_name = row['name']

    results = get_yelp_places(lat, lon)

    if results:
        for business in results.get("businesses", []):
            name = business.get('name', 'N/A')
            rating = business.get('rating', 'N/A')
            address = ', '.join(business['location'].get('display_address', [])) if 'location' in business else 'N/A'
            distance = business.get('distance', 'N/A')
            categories = business.get('categories', [])
            category_names = [cat.get('title', '') for cat in categories]
            primary_category = category_names[0] if category_names else 'N/A'

            parsed_results.append({
                'station_name': station_name,
                'business_name': name,
                'rating': rating,
                'address': address,
                'distance': distance,
                'category': primary_category
            })

# Put parsed results into a DataFrame
df_yelp_poi = pd.DataFrame(parsed_results)

# Display the first few rows of the DataFrame
print(df_yelp_poi.head())


         station_name          business_name  rating  \
0  Argenson - Château             Bustronome     4.5   
1  Argenson - Château  La Table des Oliviers     4.3   
2  Argenson - Château      Léon de Bruxelles     3.9   
3  Argenson - Château             FrogBurger     4.5   
4  Argenson - Château   Le Chalet de Neuilly     4.0   

                                             address     distance  \
0               2 avenue Kléber, 75116 Paris, France  2800.624423   
1  4 rue de l'Eglise, 92200 Neuilly Sur Seine, Fr...   419.710528   
2  95 boulevard Gouvion Saint Cyr, 75017 Paris, F...  1845.469007   
3  150 avenue Charles de Gaulle, 92200 Neuilly-su...   478.621497   
4  14 Rue du Commandant Pilot, 92200 Neuilly-sur-...  1295.350109   

        category  
0         French  
1  Mediterranean  
2        Seafood  
3        Burgers  
4         Fondue  


Compare results


In [23]:
import pandas as pd

# Create an empty list to hold each row of comparison data
comparison_data = []

# Iterate over the rows of df_yelp_poi and compare with df_foursquare
for index, row in df_yelp_poi.iterrows():
    station_name = row['station_name']
    yelp_business_name = row['business_name']
    yelp_rating = row['rating']
    yelp_distance = row['distance']

    # Find corresponding rows in Foursquare DataFrame (multiple results for each station possible)
    foursquare_rows = df_foursquare[df_foursquare['station_name'] == station_name]
    
    if not foursquare_rows.empty:
        for _, foursquare_row in foursquare_rows.iterrows():
            foursquare_business_name = foursquare_row['venue_name']
            foursquare_rating = foursquare_row.get('venue_rating', 'N/A')  # Safe access
            foursquare_distance = foursquare_row.get('venue_distance', 'N/A')  # Safe access

            # Append data to the list as a dictionary
            comparison_data.append({
                "station_name": station_name,
                "yelp_business_name": yelp_business_name,
                "foursquare_business_name": foursquare_business_name,
                "yelp_rating": yelp_rating,
                "foursquare_rating": foursquare_rating,
                "yelp_distance": yelp_distance,
                "foursquare_distance": foursquare_distance
            })

# Convert the list of dictionaries into a DataFrame
comparison_table = pd.DataFrame(comparison_data)

# Display the comparison table
print(comparison_table)

# Optionally save to CSV
comparison_table.to_csv('yelp_vs_foursquare_comparison.csv', index=False)


                  station_name    yelp_business_name  \
0           Argenson - Château            Bustronome   
1           Argenson - Château            Bustronome   
2           Argenson - Château            Bustronome   
3           Argenson - Château            Bustronome   
4           Argenson - Château            Bustronome   
..                         ...                   ...   
745  Ney - Porte de Montmartre  La Boîte aux Lettres   
746  Ney - Porte de Montmartre  La Boîte aux Lettres   
747  Ney - Porte de Montmartre  La Boîte aux Lettres   
748  Ney - Porte de Montmartre  La Boîte aux Lettres   
749  Ney - Porte de Montmartre  La Boîte aux Lettres   

      foursquare_business_name  yelp_rating foursquare_rating  yelp_distance  \
0         Les Pieds dans l'Eau          4.5               N/A    2800.624423   
1                   Maison Wok          4.5               N/A    2800.624423   
2                        Livio          4.5               N/A    2800.624423   
3      

Which API provided you with more complete data? Provide an explanation. 

Based on the comparison result, Yelp provided more complete information. The Yelp results include details like ratings, business names, addresses, and distances. However, Foursquare only included the venue names, addresses, categories, and distances without ratings in many cases.

Yelp appears to offer more detailed information such as user ratings for businesses, which can be a crucial factor for comparison. Foursquare, while helpful for nearby places and categories, may lack the depth in some cases.

Get the top 10 restaurants according to their rating

In [24]:
# Sort the dataframe by rating in descending order and get top 10 restaurants
top_10_restaurants = df_yelp_poi.sort_values(by='rating', ascending=False).head(10)

# Display the top 10 restaurants
print(top_10_restaurants[['station_name', 'business_name', 'rating']])


                          station_name              business_name  rating
18           Mairie de Rosny-sous-Bois                        Pes     5.0
123                     Suzanne Masson                 Café Cacao     5.0
122                     Suzanne Masson  Chez Gaby, Casal Do Corvo     5.0
131            Place Charles de Gaulle                   Le Corot     5.0
108         Claude Decaen - Cannebière                   Mokonuts     5.0
44   Abbeville - Faubourg Poissonnière                      ASPIC     5.0
87         République - Auguste Perret               Le Médaillon     5.0
88         République - Auguste Perret              Augusta Pizza     5.0
76             Lecourbe - Jean Maridor                Neige d'Eté     4.9
7               Saint Lambert - Blomet                Neige d'Eté     4.9
